In [1]:
import xarray as xr
import numpy as np
import sys, os, glob
import time
from scipy.stats import weibull_min
import dask.distributed as dd

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

os.chdir(root_dir)

from data_processing.libraries import mean_statistics, std_statistics, wind_power_density, weibull , weibull_statistics, wind_power_density, turbine_power, solar_power
from data_processing.libraries import regional_extraction

root_dir = '/media/harish/SSD_4TB/EU_SCORES_project'
scripts_dir = f'{root_dir}/scripts'
sys.path.append(scripts_dir)

from analysis.plotters import map_plotter, all_sample_points, variability_plotter, months, colors, calculate_color_levels, CERRA_target_grids
from data_processing.libraries import longitude_convert_0_to_360, find_nearest_indice, compute_statistics

In [38]:
client.close()
cluster.close()

NameError: name 'client' is not defined

In [ ]:
import dask.distributed as dd
cluster = dd.LocalCluster(n_workers=n_workers, threads_per_worker=2, memory_limit='2GB',dashboard_address=f':{dashboard_port}')
client = dd.Client(cluster)
print(client)

<Client: 'tcp://127.0.0.1:37427' processes=4 threads=8, memory=7.45 GiB>


In [35]:
run = 'New_runs'
case = 'Germany_coast'
level = '10'

run_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/variablewise_files'
target_dir=f'{root_dir}/WRFV4.4/EU_SCORES/{run}/{case}/Postprocessed/statistics_files/ws_{level}'
os.makedirs(target_dir,exist_ok=True)

In [51]:
time_coord='Time'
south_north='south_north'
west_east='west_east'
time_scale='season'
dim='Time'

target_file = f'{target_dir}/weibull.nc'
if os.path.exists(target_file):
    with xr.open_dataset(target_file) as target_ds:
        target_ds = target_ds.load()
else:
    target_ds = xr.Dataset()
target_ds

<xarray.Dataset>
Dimensions:         (west_east: 63, south_north: 63, parameter: 2, season: 4,
                     hour: 24, month: 12, year: 31)
Coordinates:
  * season          (season) object 'DJF' 'JJA' 'MAM' 'SON'
  * south_north     (south_north) int64 0 1 2 3 4 5 6 7 ... 56 57 58 59 60 61 62
  * west_east       (west_east) int64 0 1 2 3 4 5 6 7 ... 56 57 58 59 60 61 62
    XLONG           (south_north, west_east) float32 7.657 7.665 ... 8.137 8.145
    XLAT            (south_north, west_east) float32 54.06 54.06 ... 54.34 54.34
  * parameter       (parameter) object 'shape' 'scale'
  * hour            (hour) int64 0 1 2 3 4 5 6 7 8 ... 16 17 18 19 20 21 22 23
  * month           (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * year            (year) int64 1990 1991 1992 1993 ... 2017 2018 2019 2020
Data variables:
    season_values   (west_east, south_north, parameter, season) float64 2.613...
    hourly_values   (west_east, south_north, parameter, hour) float64 2.329 ....
    monthly_values  (west_east, south_north, parameter, month) float64 2.647 ...
    yearly_values   (west_east, south_north, parameter, year) float64 2.185 ....
    overall_values  (west_east, south_north, parameter) float64 2.269 ... 8.605